In [14]:
import glob
import shutil
from osgeo import gdal
import cv2
import numpy as np
import skimage.transform as transform

In [3]:
geotif_dsm_file =  '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/mapping/orthomosaics/observation088_dsm.tif'
geotif_image_file = '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/mapping/orthomosaics/observation088_transparent_mosaic_group1.tif'
dsm_gtif = gdal.Open(geotif_dsm_file)
image_gtif = gdal.Open(geotif_image_file)

In [4]:
dsm = dsm_gtif.GetRasterBand(1).ReadAsArray()
dsm_ave = np.min(dsm[dsm!= -10000])
dsm = np.where(dsm == -10000, dsm_ave, dsm)
elevation_r = dsm

bands = []
for band_num in range(1, 4):
    srcband = image_gtif.GetRasterBand(band_num)
    a = srcband.ReadAsArray()
    bands.append(a)
image_map = np.stack(bands, 2)
plt.figure()
plt.imshow(image_map)

In [30]:
background_map_image= transform.rescale(image_map, .1, multichannel=True, mode='reflect', anti_aliasing=True)

In [ ]:
def fig2data ( fig ):
    """
    @brief Convert a Matplotlib figure to a 4D numpy array with RGBA channels and return it
    @param fig a matplotlib figure
    @return a numpy 3D array of RGBA values
    """
    # draw the renderer
    fig.canvas.draw()
 
    # Get the RGBA buffer from the figure
    w,h = fig.canvas.get_width_height()
    buf = np.fromstring(fig.canvas.tostring_argb(), dtype=np.uint8)
    buf.shape = (h, w, 4)
 
    # canvas.tostring_argb give pixmap in ARGB mode. Roll the ALPHA channel to have it in RGBA mode
    buf = np.roll(buf, 3, axis=2)
    return cv2.cvtColor(buf, cv2.COLOR_RGBA2BGR)

def job(feed_dict):

    idx = feed_dict['idx']
    frame = feed_dict['frame']
    positions = feed_dict['positions']
#     frame = frame[40:160,20:140,1]
    plt.style.use('dark_background')
    fig, (ax1) = plt.subplots(1, 1, figsize=(10,10))
    ax1.imshow(frame, cmap='gray', interpolation='none', aspect='auto')
    for position_ind in range(positions.shape[0]):
        ax1.plot(positions[position_ind][0], positions[position_ind][1])
    ax1.axis('off')
    output_frame = fig2data(fig)
    plt.close(fig)
    
    return output_frame

fourcc = cv2.VideoWriter_fourcc(*'H264')
out = cv2.VideoWriter('map_animation.mp4', fourcc, 30.0, (720,360))

batch_size=50
start_idx = 0
end_idx = 500
frame_indexes = np.arange(start_idx, end_idx)

test_positions = np.random.rand(end_idx, 10, 2) * background_map_image.shape[:2]

main_loop_idx = 0
pool = Parallel(-1)
while main_loop_idx*batch_size < end_idx-start_idx:
#for i in range(1):
    feed_dicts = [{'frame':background_map_image, 'idx':frame_indexes[idx], 'positions': test_positions[idx]} for idx in zip(np.arange(main_loop_idx*batch_size, (main_loop_idx+1)*batch_size))]
    output_frames = pool.process(job, feed_dicts)
    out.write_batch(output_frames)
    print(main_loop_idx*batch_size)
    main_loop_idx += 1
pool.close()    
out.release()   

In [36]:
r = np.random.rand(500, 10, 2) 

In [42]:
r_new = r * background_map_image.shape[:2]

In [48]:
fourcc = cv2.VideoWriter_fourcc(*'H264')
out = cv2.VideoWriter('map_animation.mp4', fourcc, 30.0, (720,360))
out.release()